In [1]:
import pandas as pd

housing_data = pd.read_csv("NY-House-Dataset.csv")

print("New York Housing Market dataset:")
print(housing_data.head())

rat_sightings_data = pd.read_csv("Rat_Sightings.csv")

print("\nNYC Rat Sightings dataset:")
print(rat_sightings_data.head())


New York Housing Market dataset:
                                         BROKERTITLE                TYPE  \
0        Brokered by Douglas Elliman  -111 Fifth Ave      Condo for sale   
1                                Brokered by Serhant      Condo for sale   
2                             Brokered by Sowae Corp      House for sale   
3                                Brokered by COMPASS      Condo for sale   
4  Brokered by Sotheby's International Realty - E...  Townhouse for sale   

       PRICE  BEDS       BATH  PROPERTYSQFT  \
0     315000     2   2.000000        1400.0   
1  195000000     7  10.000000       17545.0   
2     260000     4   2.000000        2015.0   
3      69000     3   1.000000         445.0   
4   55000000     7   2.373861       14175.0   

                                             ADDRESS                    STATE  \
0                               2 E 55th St Unit 803       New York, NY 10022   
1  Central Park Tower Penthouse-217 W 57th New Yo...       New Yo

**New York Housing Market dataset:**
- There is a wide range of housing types listed, including condos, houses, and townhouses.
- Prices vary significantly, with some properties listed for millions of dollars and others for much less.
- The number of bedrooms and bathrooms also varies, indicating differences in property sizes and types.
- Geographical coordinates (latitude and longitude) are provided for each property, allowing for spatial analysis.

**NYC Rat Sightings dataset:**
- Rat sightings occur across various types of locations, including residential buildings, commercial buildings, and mixed-use buildings.
- There are missing values in the "Incident Address" column, indicating that not all sightings are associated with specific addresses.
- Rat sightings are reported across different zip codes and neighborhoods in New York City.
- The dataset includes geographical coordinates (latitude and longitude) for each sighting, facilitating spatial analysis and mapping.

**Observations:**
- There may be a relationship between the type of location (e.g., residential, commercial) and the frequency of rat sightings.
- It's possible that certain neighborhoods or zip codes have a higher incidence of rat sightings compared to others.
- There might be spatial correlations between areas with high house prices and areas with a high frequency of rat sightings, but further analysis is needed to confirm this.


**What feature/s would you like to be able to predict?** The geographical coordinates (latitude and longitude) of both house listings and rat sightings are crucial for spatial analysis. Mapping the distribution of house prices and rat sightings can help identify spatial patterns and correlations between the two features.

Geographical coordinates provide precise location information for each data point, allowing us to visualize the spatial distribution of house prices and rat sightings on a map. By plotting these coordinates, we can observe clusters or hotspots of high and low values for each feature across different neighborhoods or areas of New York City.

